In [22]:
pip install imblearn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/238.4 kB ? eta -:--:--
   ------ --------------------------------- 41.0/238.4 kB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 163.8/238.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 238.4/238.4 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install torch

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
print("Is GPU available:", torch.cuda.is_available())

Is GPU available: False


In [23]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import re

In [24]:
df = pd.read_csv("../data/recipes.csv")

In [25]:
print(df['recipe_name'].value_counts())

recipe_name
Apple-Cranberry Crisp                            5
Awesome Sausage, Apple and Cranberry Stuffing    5
Apple-Cranberry Crostada                         4
Apple Pie by Grandma Ople                        4
Waldorf Salad                                    4
                                                ..
Duck Sauce                                       1
Christmas Plum Pudding                           1
Plum Pudding                                     1
My Mother-in-Law's Plum Bread                    1
Chef John's Chocolate Energy Bars                1
Name: count, Length: 961, dtype: int64


In [5]:
print(f"Dataset shape: {df.shape}")

Dataset shape: (1090, 15)


In [26]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    return text

df['cleaned_ingredients'] = df['ingredients'].apply(clean_text)

In [27]:
print("Class Distribution (Before Balancing):")
print(df['recipe_name'].value_counts())

Class Distribution (Before Balancing):
recipe_name
Apple-Cranberry Crisp                            5
Awesome Sausage, Apple and Cranberry Stuffing    5
Apple-Cranberry Crostada                         4
Apple Pie by Grandma Ople                        4
Waldorf Salad                                    4
                                                ..
Duck Sauce                                       1
Christmas Plum Pudding                           1
Plum Pudding                                     1
My Mother-in-Law's Plum Bread                    1
Chef John's Chocolate Energy Bars                1
Name: count, Length: 961, dtype: int64


In [28]:
vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
X = vectorizer.fit_transform(df['cleaned_ingredients'])
y = df['recipe_name']

In [45]:
min_samples = 2  # Minimum samples required in a class
class_counts = pd.Series(y).value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
mask = y.isin(valid_classes)

In [46]:
y = y[mask]

In [47]:
X = X[mask.to_numpy()]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
def resample_with_error_handling(X_train, y_train):
    """
    Resample the dataset using SMOTEENN with custom configurations. If errors are encountered,
    problematic classes will be dynamically removed.
    """
    smote = SMOTE(k_neighbors=1, random_state=42)  # Use k_neighbors=1 to handle minority classes
    enn = EditedNearestNeighbours(n_neighbors=1)  # Use n_neighbors=1 for the ENN step
    smote_enn = SMOTEENN(smote=smote, enn=enn, random_state=42)

    while True:
        try:
            # Attempt to resample
            X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
            return X_resampled, y_resampled
        except ValueError as e:
            # Handle the ValueError caused by rare classes
            if "Expected n_neighbors <= n_samples_fit" in str(e):
                print("Rare classes detected. Removing problematic classes...")
                # Identify and remove classes with fewer than 2 samples
                class_counts = pd.Series(y_train).value_counts()
                valid_classes = class_counts[class_counts >= 2].index
                mask = y_train.isin(valid_classes)

                # Update X_train and y_train
                y_train = y_train[mask]
                X_train = X_train[mask.to_numpy()]
            else:
                # Raise other unexpected errors
                raise e

# Apply resampling with error handling
X_resampled, y_resampled = resample_with_error_handling(X_train, y_train)

# Step 4: Print summary of resampling
print("Data resampling with SMOTEENN complete!")
print(f"Original training dataset size: {len(y_train)}")
print(f"Resampled training dataset size: {len(y_resampled)}")
print(f"Number of unique classes after resampling: {len(pd.Series(y_resampled).value_counts())}")

Rare classes detected. Removing problematic classes...
Data resampling with SMOTEENN complete!
Original training dataset size: 110
Resampled training dataset size: 170
Number of unique classes after resampling: 34


In [58]:
train_labels = set(y_resampled)

# Create a boolean mask
test_mask = y_test.isin(train_labels)

# Use the boolean mask to filter y_test (directly works with pandas Series)
y_test_filtered = y_test[test_mask]

# Convert the boolean mask to a numpy array and use it to filter the sparse matrix X_test
X_test_filtered = X_test[test_mask.to_numpy()]

In [59]:
catboost_model = CatBoostClassifier(
    iterations=500,
        learning_rate=0.1,
        depth=6,
        loss_function='MultiClass',
        eval_metric='Accuracy',
        random_seed=42,
        verbose=10
)

In [60]:
catboost_model.fit(X_resampled, y_resampled, eval_set=(X_test_filtered, y_test_filtered))

0:	learn: 0.2941176	test: 0.2000000	best: 0.2000000 (0)	total: 35.8ms	remaining: 17.9s
10:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 122ms	remaining: 5.42s
20:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 204ms	remaining: 4.64s
30:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 292ms	remaining: 4.42s
40:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 378ms	remaining: 4.23s
50:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 458ms	remaining: 4.03s
60:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 555ms	remaining: 4s
70:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 646ms	remaining: 3.9s
80:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 733ms	remaining: 3.79s
90:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 829ms	remaining: 3.72s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (5)	total: 916ms	remaining: 3.62s
110:	learn: 1.0000000	test: 1.0000000	best: 1.

In [69]:
y_pred = catboost_model.predict(X_test_filtered)

# Check accuracy
accuracy = accuracy_score(y_test_filtered, y_pred)
print("CatBoost Accuracy:", accuracy)

# Optional: Print a classification report for more detailed metrics
print("Classification Report:\n", classification_report(y_test_filtered, y_pred))

CatBoost Accuracy: 1.0
Classification Report:
                                    precision    recall  f1-score   support

             Amazing Apple Butter       1.00      1.00      1.00         1
                      Apple Crisp       1.00      1.00      1.00         1
                  Apple Crumb Pie       1.00      1.00      1.00         2
                Apple Pie Filling       1.00      1.00      1.00         1
         Apple-Cranberry Crostada       1.00      1.00      1.00         1
             Basic Fruit Smoothie       1.00      1.00      1.00         1
         Canned Apple Pie Filling       1.00      1.00      1.00         2
                   Caramel Apples       1.00      1.00      1.00         1
     Debbie's Amazing Apple Bread       1.00      1.00      1.00         1
                 Easy Apple Cider       1.00      1.00      1.00         1
      Easy Apple Cinnamon Muffins       1.00      1.00      1.00         2
Easy Apple Crisp with Oat Topping       1.00      1.

In [ ]:
# catboost_model = CatBoostClassifier(
#     iterations=100,
#     depth=6,
#     learning_rate=0.1,
#     l2_leaf_reg=10,  # Added regularization
#     border_count=32,  # Reduced border splits
#     task_type='CPU',
#     early_stopping_rounds=10,
#     thread_count=4,  # Limit to 4 threads
#     verbose=10  # Disable logging
# )

In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test_filtered)

logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_resampled_scaled, y_resampled)

logistic_y_pred = logistic_model.predict(X_test_scaled)
print("Logistic Regression Accuracy:", accuracy_score(y_test_filtered, logistic_y_pred))


Logistic Regression Accuracy: 1.0


In [68]:
svc_model = SVC(kernel='linear', probability=True, random_state=42)
svc_model.fit(X_resampled_scaled, y_resampled)

svc_y_pred = svc_model.predict(X_test_scaled)
print("SVC Accuracy:", accuracy_score(y_test_filtered, svc_y_pred))

SVC Accuracy: 1.0


In [62]:
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [63]:
svc_model.fit(X_train, y_train)

SVC(kernel='linear')

In [64]:
models = {
    "CatBoost": catboost_model,
    "Logistic Regression": logistic_model,
    "SVC": svc_model
}

In [65]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.2f}")

CatBoost Accuracy: 0.89
Logistic Regression Accuracy: 0.71
SVC Accuracy: 0.86
